In [2]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import re

In [3]:
#show evrything in dataframe
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [4]:
#open empty json file
# Define the file path
file_path = 'Automated_SIPM_Det.json'

# Create an empty dictionary
data = {}

# Write the empty dictionary to the JSON file
with open(file_path, 'w') as f:
    json.dump(data, f)


In [5]:
#open the corresponding file
#always choose .tsv file
df = pd.read_csv('LAr_SIPmsIB_OB_2nd_immersion_Dec24.tsv', sep='\t')

In [6]:
def format_to_orca(df: pd.DataFrame) -> pd.DataFrame:
    # Replace NaN values with '-'
    df = df.fillna('-')
    # Retain only columns up to and including 'HV Channel'
    if 'Voltage' in df.columns:
        df = df.iloc[:, :df.columns.get_loc('Voltage') + 1]
    # Rename specific columns
    rename_mapping = {' ': 'Det_name', 'HV Flange.1': 'HV Flange Pos'}
    df = df.rename(columns=rename_mapping)
    # Remove rows that contain 'String' in any column
    df = df[~df.apply(lambda row: row.astype(str).str.contains('String', na=False).any(), axis=1)]
    # Remove rows where all values are '-'
    df = df[~(df == '-').all(axis=1)]
    # Reset index
    df = df.reset_index(drop=True)
    # Limit to first 60 rows
    return df.iloc[:60]

In [7]:
#df=format_to_orca(df)

In [8]:
def cleandf(df):
  columns_to_drop = ['Data ID ', 'DSUB ID']
  # Drop only existing columns to avoid errors
  df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])
  df = df.fillna('--')  # Replace NaN with an empty string
  df = df.replace(['-','--','---'], '--')  # Replace '--', '-', and '---' with an empty string
  # Remove rows where all values are '-'
  df = df[~(df == '--').all(axis=1)]
  return df

In [9]:
#df=cleandf(df)

In [10]:
def add_loc_FormatSiPM(df):
  df_i= df[df["Fiber ID"].str.contains("IB", na=False)].copy()

  # Sort by "angle" in ascending order
  df_i = df_i.sort_values(by="Angle")

  # Create a new column increasing linearly with "angle"
  df_i["loc"] = pd.factorize(df_i["Angle"])[0]
  df_i["loc"]=df_i["loc"]+1

  df_i = df_i.sort_values(by="Fiber ID")
  df_i

  df_o= df[df["Fiber ID"].str.contains("OB", na=False)].copy()

  # Sort by "angle" in ascending order
  df_o = df_o.sort_values(by="Angle")

  # Create a new column increasing linearly with "angle"
  df_o["loc"] = pd.factorize(df_o["Angle"])[0]
  df_o["loc"]=df_o["loc"]+1
  df_o = df_o.sort_values(by="Fiber ID")
  df_o
  df_t = pd.concat([df_i, df_o], axis=0)
  #format sipm id
  df_t['SiPM ID'] = df_t['SiPM ID'].astype(str)
  # Apply formatting only if the ID doesn't start with 'S'
  df_t['SiPM ID'] = df_t['SiPM ID'].apply(lambda x: 'S' + x.zfill(2) if not x.startswith('S') else x)
  #add sipm type as inner and outer barrel
  df_t['SiPM Type'] = df_t['Fiber ID'].str[:2]
  return df_t

In [11]:
def process_entry(entry):
    if isinstance(entry, str):  # Ensure it's a string
        parts = entry.split('-') if '-' in entry else entry.split('_')
        if len(parts) < 2:
            return None  # Handle unexpected formats

        prefix, num = parts[0], int(parts[1])

        if prefix == 'IB':
            return '3' if num % 2 == 0 else '2'  # IB cases
        elif prefix == 'OB ':
            return '4' if num % 2 == 0 else '1'  # OB cases

    return None  # Default case for missing/invalid values

In [12]:
# add board id with mapping
# Define mapping dictionary
mapping = {
    (2, 0): '0x200',
    (2, 1): '0x210',
    (2, 2): '0x220',
    (2, 3): '0x230',
    (2, 4): '0x240',
    (2, 5): '0x250',
    (2, 6): '0x260',
    (2, 7): '0x270',
    (2, 8): '0x280',
    (2, 9): '0x290',
    (2, 10): '0x2A0',
    (2, 11): '0x2B0',
    (2, 12): '0x2C0'
    }

In [13]:
def add_entry_to_json(df: pd.DataFrame, json_file: str):
    """
    Adds entries from a DataFrame to a JSON file following the specified format.

    Parameters:
    df (pd.DataFrame): DataFrame containing the new data.
    json_file (str): Path to the JSON file.
    """

    # Load existing JSON data or initialize an empty dictionary
    try:
        with open(json_file, 'r') as file:
            data = json.load(file)
    except (FileNotFoundError, json.JSONDecodeError):
        data = {}

    # Iterate over the DataFrame rows and add entries to JSON
    for _, row in df.iterrows():
        key = row['SiPM ID']  # Unique identifier for the entry (e.g., 'BF862_1', 'BF862_2', etc.)

        data[key] = {
            "system": str("spm"),
            "daq": {
                "board_ch": str(row.get("Ch", "--")),
                "adc_serial": str(row.get("ADC Serial", "--")),
                "board_slot": str(row.get("Slot", "--")),
                "board_id": str(row.get("Flashcam Address", "--")),
                "crate": str(row.get("Crate", "--"))
            },
            "low_voltage": {
                "board_chan": str(row.get("HV Channel", "--")),
                "board_slot": str(row.get("HV Module", "--")),
                "board_B_pos": str(row.get("pos", "--")),
                "board_string": str(row.get("loc", "--")),
                "crate": str('0')
            },
            "det_type": str(row.get("SiPM Type", "--")),
        }

    # Write back to JSON file
    with open(json_file, 'w') as file:
        json.dump(data, file, indent=4)

    print(f"Entries successfully added to {json_file}")


In [14]:

# Apply the function to the DataFrame
df=format_to_orca(df)
df = cleandf(df)
df = add_loc_FormatSiPM(df)
df['pos'] = df['Fiber ID'].apply(process_entry)
df['Flashcam Address'] = df.apply(lambda row: mapping.get((row['Crate'], row['Slot']), '--'), axis=1)

In [15]:
add_entry_to_json(df,'Automated_SIPM_Det.json')

Entries successfully added to Automated_SIPM_Det.json


In [16]:
df

,Crate,Slot,Ch,FC ID,FE ID,Fiber ID,Angle,SiPM ID,Voltage,loc,SiPM Type,pos,Flashcam Address
0,2,1,4,10,B0-10,IB-01,240,S71,25.5,7,IB,2,0x210
1,2,1,5,11,B0-11,IB-02,240,S70,25.5,7,IB,3,0x210
2,2,2,2,14,B1-02,IB-03,200,S90,26.5,6,IB,2,0x220
3,2,2,3,15,B1-03,IB-04,200,S29,25.5,6,IB,3,0x220
4,2,3,0,18,B1-06,IB-05,160,S24,24.0,5,IB,2,0x230
5,2,3,1,19,B1-07,IB-06,160,S23,25.0,5,IB,3,0x230
6,2,3,4,22,B1-10,IB-07,120,S02,25.0,4,IB,2,0x230
7,2,3,5,23,B1-11,IB-08,120,S03,25.0,4,IB,3,0x230
8,2,4,0,24,B2-00,IB-09,80,S32,25.5,3,IB,2,0x240
9,2,4,1,25,B2-01,IB-10,80,S36,26.0,3,IB,3,0x240
